[**Network in Network (NiN) (Lin et al., 2013):**](https://arxiv.org/abs/1312.4400) is a pioneering approach that convolves whole neural networks patch-wise over inputs, introducing mlpconv layers to enhance feature abstraction.

![](imgs/nin.png)

> Image Source: [TowardDataScience](https://medium.com/towards-data-science/review-nin-network-in-network-image-classification-69e271e499ee)

The Network in Network (NiN) architecture enhances traditional CNNs by incorporating micro-networks within each convolutional layer. Instead of using standard convolutional filters that apply a single linear transformation, NiN replaces them with MLPConv layers, which consist of multiple 1×1 convolutions followed by non-linear activations. This design allows for greater abstraction and feature extraction at each spatial location, making NiN more powerful than traditional CNNs like AlexNet. Additionally, the architecture consists of multiple NiN blocks, where each block includes a standard convolution followed by two 1×1 convolution layers, enabling parameter efficiency while increasing representational capacity.

A key innovation in NiN is the removal of fully connected layers. Instead of using dense layers for classification, NiN applies a Global Average Pooling (GAP) layer, which averages each feature map before passing the result to the softmax classifier. This significantly reduces model parameters, minimizes overfitting, and improves generalization. The overall structure consists of three NiN blocks interleaved with max-pooling layers, followed by a final 1×1 convolution and the GAP layer. Despite its simplicity, NiN provides strong performance, particularly in reducing computational complexity and improving feature learning, making it a compelling alternative to traditional deep CNN architectures.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import utils

In [2]:
def nin_block(in_channels, out_channels, kernel_size, stride, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding), nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU(inplace=True)
    )

In [3]:
class NiN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()

        self.net = nn.Sequential(
            nin_block(3, 96, kernel_size=11, stride=4, padding=0),
            nn.MaxPool2d(3, stride=2),
            nin_block(96, 256, kernel_size=5, stride=1, padding=2),
            nn.MaxPool2d(3, stride=2),
            nin_block(256, 384, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(3, stride=2),
            nn.Conv2d(384, num_classes, kernel_size=3, stride=1, padding=1),
            nn.AdaptiveAvgPool2d((1, 1)),  # Global Average Pooling (GAP)
            nn.Flatten()
        )

        self.net.apply(utils.init_cnn)

    def forward(self, x):
        return self.net(x)

In [4]:
utils.layer_summary(NiN(num_classes=10), (1, 3, 224, 224))

Layer Name                     Layer Type              Param #         Output Shape
net.0.0                        Conv2d                    34944      (1, 96, 54, 54)
net.0.1                        ReLU                          0      (1, 96, 54, 54)
net.0.2                        Conv2d                     9312      (1, 96, 54, 54)
net.0.3                        ReLU                          0      (1, 96, 54, 54)
net.0.4                        Conv2d                     9312      (1, 96, 54, 54)
net.0.5                        ReLU                          0      (1, 96, 54, 54)
net.1                          MaxPool2d                     0      (1, 96, 26, 26)
net.2.0                        Conv2d                   614656     (1, 256, 26, 26)
net.2.1                        ReLU                          0     (1, 256, 26, 26)
net.2.2                        Conv2d                    65792     (1, 256, 26, 26)
net.2.3                        ReLU                          0     (1, 256, 

In [5]:
data = utils.CIFAR10DataLoader(batch_size=64, resize=(224, 224))
train_loader = data.get_train_loader()
test_loader = data.get_test_loader()

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NiN(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
for epoch in range(epochs):
    train_loss, train_acc = utils.train_step(train_loader, model, criterion, optimizer, device)
    test_loss, test_acc = utils.eval_step(test_loader, model, criterion, device)
    print(f"Epoch {epoch + 1}/{epochs}: Train Loss={train_loss}, Test Loss={test_loss}, Test Accuracy={test_acc}")

Epoch 1/10: Train Loss=1.8432605034859895, Test Loss=1.618095687240552, Test Accuracy=0.4062
Epoch 2/10: Train Loss=1.4822881791902625, Test Loss=1.431696512137249, Test Accuracy=0.4829
Epoch 3/10: Train Loss=1.3018485482239053, Test Loss=1.2204991897959618, Test Accuracy=0.561
Epoch 4/10: Train Loss=1.1519264766322377, Test Loss=1.1275896866610096, Test Accuracy=0.6024
Epoch 5/10: Train Loss=1.0283166048167003, Test Loss=1.058608191787817, Test Accuracy=0.633
Epoch 6/10: Train Loss=0.9345457960882455, Test Loss=0.9927069480252114, Test Accuracy=0.6577
Epoch 7/10: Train Loss=0.8538130572461107, Test Loss=0.89729250578364, Test Accuracy=0.6862
Epoch 8/10: Train Loss=0.7782667841371673, Test Loss=0.8768381749748424, Test Accuracy=0.6992
Epoch 9/10: Train Loss=0.7100458946984137, Test Loss=0.9168987486772476, Test Accuracy=0.6826
Epoch 10/10: Train Loss=0.6477723862890088, Test Loss=0.9135754909485009, Test Accuracy=0.6934
